In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import time
tf.__version__

'1.0.0'

In [2]:
# Load the data
lines = open('movie_lines.txt').read().split('\n')
conv_lines = open('movie_conversations.txt').read().split('\n')

In [3]:
# Match line id with text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5: #keeping only sentences with >5 words
        id2line[_line[0]] = _line[4]

In [4]:
# Create a list of all of the conversations' lines' ids.
convs = [ ]
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))

In [5]:
convs[:10] #previewing conversations
convs=convs[:10000]

In [6]:
# Order the sentences into questions and answers
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])

In [7]:
# Examples
limit = 0
for i in range(limit, limit+5):
    print(questions[i])
    print(answers[i])
    print " "

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.
 
Well, I thought we'd start with pronunciation, if that's okay with you.
Not the hacking and gagging and spitting part.  Please.
 
Not the hacking and gagging and spitting part.  Please.
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?
 
You're asking me out.  That's so cute. What's your name again?
Forget it.
 
No, no, it's my fault -- we didn't have a proper introduction ---
Cameron.
 


In [8]:
#removing shortened words, punctuation
def clean_text(text):

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [9]:
# Apply to questions and answers
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))

In [10]:
# Preview cleaned data
limit = 0
for i in range(limit, limit+5):
    print(clean_questions[i])
    print(clean_answers[i])
    print " "

can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again
well i thought we would start with pronunciation if that is okay with you
 
well i thought we would start with pronunciation if that is okay with you
not the hacking and gagging and spitting part  please
 
not the hacking and gagging and spitting part  please
okay then how about we try out some french cuisine  saturday  night
 
you are asking me out  that is so cute that is your name again
forget it
 
no no it is my fault  we did not have a proper introduction 
cameron
 


In [11]:
#Inspecting lengths of sentences
lengths = []
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))

lengths = pd.DataFrame(lengths, columns=['counts'])

In [12]:
lengths.describe()

,counts
count,50298.000000
mean,10.883574
std,11.909324
min,0.000000
25%,4.000000
50%,7.000000
75%,14.000000
max,498.000000


In [13]:
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

16.0
19.0
24.0
32.0
57.0


In [14]:
# Remove questions and answers that are shorter than 2 words and longer than 20 words.
min_line_length = 2
max_line_length = 20

short_questions_temp = []
short_answers_temp = []

i = 0
for question in clean_questions:
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])
    i += 1


short_questions = []
short_answers = []

i = 0
for answer in short_answers_temp:
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
    i += 1

In [15]:
# Create a dictionary
#Contains word--->word frequency
vocab = {}
for question in short_questions:
    for word in question.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
for answer in short_answers:
    for word in answer.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [16]:
#only keeping words that occure more than 10 times
threshold = 10

#Create dictionaries to provide a unique integer for each word
#This is a needed step in order to create our embeddings later
questions_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        questions_vocab_to_int[word] = word_num
        word_num += 1
        
answers_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        answers_vocab_to_int[word] = word_num
        word_num += 1

In [17]:
# Add the unique tokens to the vocabulary dictionaries.
codes = ['<PAD>','<EOS>','<UNK>','<GO>']

for code in codes:
    questions_vocab_to_int[code] = len(questions_vocab_to_int)+1
    
for code in codes:
    answers_vocab_to_int[code] = len(answers_vocab_to_int)+1

In [18]:
# Create dictionaries to map the unique integers to their respective words.
# i.e. an inverse dictionary for vocab_to_int.
questions_int_to_vocab = {v_i: v for v, v_i in questions_vocab_to_int.items()}
answers_int_to_vocab = {v_i: v for v, v_i in answers_vocab_to_int.items()}

In [19]:
# Check the length of the dictionaries.
print(len(questions_vocab_to_int))
print(len(questions_int_to_vocab))
print(len(answers_vocab_to_int))
print(len(answers_int_to_vocab))

1729
1729
1729
1729


In [20]:
# Add the end of sentence token to the end of every answer.
for i in range(len(short_answers)):
    short_answers[i] += ' <EOS>'

In [21]:
#Creating sentences of word Ids
#Add UNK tag to rare (removed from vocabulary) words

questions_int = []
for question in short_questions:
    ints = []
    for word in question.split():
        if word not in questions_vocab_to_int:
            ints.append(questions_vocab_to_int['<UNK>'])
        else:
            ints.append(questions_vocab_to_int[word])
    questions_int.append(ints)
    
answers_int = []
for answer in short_answers:
    ints = []
    for word in answer.split():
        if word not in answers_vocab_to_int:
            ints.append(answers_vocab_to_int['<UNK>'])
        else:
            ints.append(answers_vocab_to_int[word])
    answers_int.append(ints)

In [22]:
# Check the lengths
print(len(questions_int))
print(len(answers_int))

15724
15724


In [23]:
# Sort questions and answers by the length of questions.

sorted_questions = []
sorted_answers = []

for length in range(1, max_line_length+1):
    for i in enumerate(questions_int):
        if len(i[1]) == length:
            sorted_questions.append(questions_int[i[0]])
            sorted_answers.append(answers_int[i[0]])

In [24]:
def model_inputs():
    #Setting up model's variables
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return input_data, targets, lr, keep_prob

In [25]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    #Add GO tag to start of each sentence
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [26]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    #Using Multilayer Bidirectional LSTM RNN cells with dropout
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    enc_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    _, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = enc_cell,
                                                   cell_bw = enc_cell,
                                                   sequence_length = sequence_length,
                                                   inputs = rnn_inputs, 
                                                   dtype=tf.float32)
    return enc_state

In [27]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob, batch_size):
    #training part of decoder
    #using attention to handle context in long sentences
    #getting as input the encoder's output
    
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = \
            tf.contrib.seq2seq.prepare_attention(attention_states,
                                                 attention_option="bahdanau",
                                                 num_units=dec_cell.output_size)
    
    train_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                     att_keys,
                                                                     att_vals,
                                                                     att_score_fn,
                                                                     att_construct_fn,
                                                                     name = "attn_dec_train")
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                              train_decoder_fn, 
                                                              dec_embed_input, 
                                                              sequence_length, 
                                                              scope=decoding_scope)
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)
    return output_fn(train_pred_drop)

In [28]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob, batch_size):
    #Similar to decoder used for training, just without adding the dropout
    #Also, since we are predicting answers, we have no labels
    #Same embedding parameters (only) used as for training
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = \
            tf.contrib.seq2seq.prepare_attention(attention_states,
                                                 attention_option="bahdanau",
                                                 num_units=dec_cell.output_size)
    
    infer_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_inference(output_fn, 
                                                                         encoder_state[0], 
                                                                         att_keys, 
                                                                         att_vals, 
                                                                         att_score_fn, 
                                                                         att_construct_fn, 
                                                                         dec_embeddings,
                                                                         start_of_sequence_id, 
                                                                         end_of_sequence_id, 
                                                                         maximum_length, 
                                                                         vocab_size, 
                                                                         name = "attn_dec_inf")
    infer_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                                infer_decoder_fn, 
                                                                scope=decoding_scope)
    
    return infer_logits

In [29]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, vocab_to_int, keep_prob, batch_size):
    #The whole decoding layer
    
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        dec_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
        
        weights = tf.truncated_normal_initializer(stddev=0.1)
        biases = tf.zeros_initializer()
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, 
                                                                vocab_size, 
                                                                None, 
                                                                scope=decoding_scope,
                                                                weights_initializer = weights,
                                                                biases_initializer = biases)

        train_logits = decoding_layer_train(encoder_state, 
                                            dec_cell, 
                                            dec_embed_input, 
                                            sequence_length, 
                                            decoding_scope, 
                                            output_fn, 
                                            keep_prob, 
                                            batch_size)
        decoding_scope.reuse_variables()
        infer_logits = decoding_layer_infer(encoder_state, 
                                            dec_cell, 
                                            dec_embeddings, 
                                            vocab_to_int['<GO>'],
                                            vocab_to_int['<EOS>'], 
                                            sequence_length - 1, 
                                            vocab_size,
                                            decoding_scope, 
                                            output_fn, keep_prob, 
                                            batch_size)

    return train_logits, infer_logits

In [30]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, answers_vocab_size, 
                  questions_vocab_size, enc_embedding_size, dec_embedding_size, rnn_size, num_layers, 
                  questions_vocab_to_int):
    
    #Sequence to sequence model
    #Encoding + decoding layers connected
    #Embeddings implemented 
    
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, 
                                                       answers_vocab_size+1, 
                                                       enc_embedding_size,
                                                       initializer = tf.random_uniform_initializer(0,1))
    enc_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob, sequence_length)

    dec_input = process_encoding_input(target_data, questions_vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.random_uniform([questions_vocab_size+1, dec_embedding_size], 0, 1))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    train_logits, infer_logits = decoding_layer(dec_embed_input, 
                                                dec_embeddings, 
                                                enc_state, 
                                                questions_vocab_size, 
                                                sequence_length, 
                                                rnn_size, 
                                                num_layers, 
                                                questions_vocab_to_int, 
                                                keep_prob, 
                                                batch_size)
    return train_logits, infer_logits


In [31]:
# Set the Hyperparameters
epochs = 6
batch_size = 64
rnn_size = 512
num_layers = 2
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.005
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.5

In [32]:
# Model inputs    
input_data, targets, lr, keep_prob = model_inputs()
# Sequence length is max sentence length
sequence_length = tf.placeholder_with_default(max_line_length, None, name='sequence_length')
input_shape = tf.shape(input_data)

# Create the training and inference predictions (aka logits)
train_logits, inference_logits = seq2seq_model(
    tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(answers_vocab_to_int), 
    len(questions_vocab_to_int), encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, 
    questions_vocab_to_int)

with tf.name_scope("optimization"):
    # Weighted softmax cross entropy  
    cost = tf.contrib.seq2seq.sequence_loss(
        train_logits,
        targets,
        tf.ones([input_shape[0], sequence_length]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)

    # Gradient Clipping
    #getting the gradient values from the optimizer
    gradients = optimizer.compute_gradients(cost)
    #normalize gradients to be in our boundaries
    capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
    #putting gradients back to the optimizer
    train_op = optimizer.apply_gradients(capped_gradients)

In [33]:
def pad_sentence_batch(sentence_batch, vocab_to_int):
    #add PAD tag to sentences with lebgth smaller than our max length
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [34]:
def batch_data(questions, answers, batch_size):
    #Generator to get the batches
    for batch_i in range(0, len(questions)//batch_size):
        start_i = batch_i * batch_size
        questions_batch = questions[start_i:start_i + batch_size]
        answers_batch = answers[start_i:start_i + batch_size]
        pad_questions_batch = np.array(pad_sentence_batch(questions_batch, questions_vocab_to_int))
        pad_answers_batch = np.array(pad_sentence_batch(answers_batch, answers_vocab_to_int))
        yield pad_questions_batch, pad_answers_batch

In [35]:
train_valid_split = int(len(sorted_questions)*0.15)

# Split the questions and answers into train and test
train_questions = sorted_questions[train_valid_split:]
train_answers = sorted_answers[train_valid_split:]

valid_questions = sorted_questions[:train_valid_split]
valid_answers = sorted_answers[:train_valid_split]

print(len(train_questions))
print(len(valid_questions))

13366
2358


In [36]:
display_step = 100 # Check training loss after every 100 batches
stop_early = 0 
stop = 5 # If the validation loss does decrease in 5 consecutive checks, stop training
validation_check = ((len(train_questions))//batch_size//2)-1 # Modulus for checking validation loss
total_train_loss = 0 # Record the training loss for each display step
summary_valid_loss = [] # Record the validation loss for saving improvements in the model

graph = tf.Graph().as_default()
saver = tf.train.Saver()  # create the saver after the graph
with tf.Session() as sess:  # Session object
                    
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(1, epochs+1):
        for batch_i, (questions_batch, answers_batch) in enumerate(
            batch_data(train_questions, train_answers, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: questions_batch,
                 targets: answers_batch,
                 lr: learning_rate,
                 sequence_length: answers_batch.shape[1],
                 keep_prob: keep_probability})

            total_train_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(train_questions) // batch_size, 
                              total_train_loss / display_step, 
                              batch_time*display_step))
                total_train_loss = 0

            if batch_i % validation_check == 0 and batch_i > 0:
                total_valid_loss = 0
                start_time = time.time()
                for batch_ii, (questions_batch, answers_batch) in \
                        enumerate(batch_data(valid_questions, valid_answers, batch_size)):
                    valid_loss = sess.run(
                    cost, {input_data: questions_batch,
                           targets: answers_batch,
                           lr: learning_rate,
                           sequence_length: answers_batch.shape[1],
                           keep_prob: 1})
                    total_valid_loss += valid_loss
                end_time = time.time()
                batch_time = end_time - start_time
                avg_valid_loss = total_valid_loss / (len(valid_questions) / batch_size)
                print('Valid Loss: {:>6.3f}, Seconds: {:>5.2f}'.format(avg_valid_loss, batch_time))

                # Reduce learning rate within limit
                learning_rate *= learning_rate_decay
                if learning_rate < min_learning_rate:
                    learning_rate = min_learning_rate

                summary_valid_loss.append(avg_valid_loss)
                if avg_valid_loss <= min(summary_valid_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver.save(sess, './my-model') #if we see an impovement, we save the model

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break

        if stop_early == stop:
            print("Stopping Training.")
            break


        

Epoch   1/6 Batch    0/208 - Loss:  0.075, Seconds: 316.69
Epoch   1/6 Batch  100/208 - Loss:  2.979, Seconds: 343.48
Valid Loss:  2.176, Seconds: 33.98
New Record!
Epoch   1/6 Batch  200/208 - Loss:  2.220, Seconds: 422.78
Valid Loss:  2.130, Seconds: 27.78
New Record!
Epoch   2/6 Batch    0/208 - Loss:  0.173, Seconds: 240.66
Epoch   2/6 Batch  100/208 - Loss:  2.089, Seconds: 301.72
Valid Loss:  2.044, Seconds: 25.55
New Record!
Epoch   2/6 Batch  200/208 - Loss:  2.091, Seconds: 450.88
Valid Loss:  2.013, Seconds: 25.30
New Record!
Epoch   3/6 Batch    0/208 - Loss:  0.163, Seconds: 269.71
Epoch   3/6 Batch  100/208 - Loss:  1.978, Seconds: 332.58
Valid Loss:  1.952, Seconds: 28.75
New Record!
Epoch   3/6 Batch  200/208 - Loss:  2.009, Seconds: 464.90
Valid Loss:  1.943, Seconds: 28.99
New Record!
Epoch   4/6 Batch    0/208 - Loss:  0.158, Seconds: 468.98
Epoch   4/6 Batch  100/208 - Loss:  1.922, Seconds: 457.48
Valid Loss:  1.909, Seconds: 28.90
New Record!
Epoch   4/6 Batch  200

In [37]:
def question_to_seq(question, vocab_to_int):
    #preprossesing user's question
    question = clean_text(question)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in question.split()]

In [38]:
counter=-1
#testing for 100 questions
while counter<100:
    counter+=1
    graph = tf.Graph().as_default() #creating default graph
    saver = tf.train.Saver()
    with tf.Session() as sess:  # your session object

        saver.restore(sess, "./my-model")
        # Restoring our best model


        # Random input question
        random = np.random.choice(len(short_questions))
        input_question = short_questions[random]

        # Prepare the question
        input_question = question_to_seq(input_question, questions_vocab_to_int)

        # Padding
        input_question = input_question + [questions_vocab_to_int["<PAD>"]] * (max_line_length - len(input_question))
        # Add empty questions so as to fill the batch
        batch_shell = np.zeros((batch_size, max_line_length))
        # User's input question
        batch_shell[0] = input_question    

        # Run the model with the input question
        answer_logits = sess.run(inference_logits, {input_data: batch_shell, 
                                                    keep_prob: 0.3})[0]

        # Remove the padding from the Question and Answer
        pad_q = questions_vocab_to_int["<PAD>"]
        pad_a = answers_vocab_to_int["<PAD>"]
        
       
        print('\nQuestion: '), str([questions_int_to_vocab[i] for i in input_question if i != pad_q])
        print('Answer'), str([answers_int_to_vocab[i] for i in np.argmax(answer_logits, 1) if i != pad_a])



Question:  ['friend', 'of', 'yours', 'did', 'you', 'help', 'him', 'out', 'of', 'a', '<UNK>']
Answer ['i', 'do', 'not', 'know', 'i', 'am', 'not', 'to', '<UNK>', '<EOS>']

Question:  ['you', 'call', 'that', 'john', '<UNK>', '<UNK>', 'shit', 'dancing', 'i', 'would', 'not', 'dance', 'like', 'that', 'in', 'private', 'if', 'you', 'paid', 'me']
Answer ['i', 'am', 'not', '<UNK>', '<EOS>', 'four', 'four', 'four', 'four', 'four']

Question:  ['well', 'we', 'will', 'see', 'you', 'later', 'bob']
Answer ['i', 'am', 'not', 'going', 'to', '<UNK>', '<EOS>', 'four', 'four', 'four']

Question:  ['what', 'get', 'away', '<UNK>']
Answer ['i', 'am', 'not', '<UNK>', '<EOS>', 'four', 'four', 'four', 'four']

Question:  ['<UNK>', 'for', 'the', 'rest', 'of', 'the', 'year']
Answer ['i', 'am', 'not', '<UNK>', '<EOS>', 'four', 'four', 'four', 'four', 'four', 'four']

Question:  ['i', 'have', 'been', 'given', 'nothing', 'god', 'makes', 'men', 'what', 'they', 'are']
Answer ['i', 'am', '<UNK>', '<EOS>', 'four', 'fou


Question:  ['try', 'again']
Answer ['i', 'do', 'not', 'know', 'i', 'do', 'not', 'have', 'to', '<UNK>', '<EOS>']

Question:  ['no', 'i', 'am', 'serious', 'that', '<UNK>', 'got', 'exactly', 'what', 'she', '<UNK>', 'yes']
Answer ['i', 'am', 'not', '<UNK>', '<EOS>', 'four', 'four', 'four', 'four', 'four']

Question:  ['what', 'do', 'i', 'get']
Answer ['i', 'am', 'not', '<UNK>', '<EOS>', 'four', 'four', 'four', 'four']

Question:  ['do', 'not', 'worry', 'it', 'is', 'really', '<UNK>', 'this', 'time', 'of', 'year']
Answer ['i', 'am', 'not', 'afraid', '<EOS>', 'four', 'four', 'four', 'four', 'four']

Question:  ['what', 'are', 'you', 'doing', 'here']
Answer ['i', 'do', 'not', 'know', '<EOS>', 'four', 'four', 'four', 'four', 'four', 'four', 'four']

Question:  ['just', 'place', 'an', '<UNK>', 'in', 'the', '<UNK>', 'place', 'on', 'the', '<UNK>']
Answer ['i', 'do', 'not', 'know', 'i', 'am', '<UNK>', '<EOS>', 'four']

Question:  ['i', 'would', 'not', 'know']
Answer ['i', 'am', 'not', '<UNK>', '<U